In [1]:
import torch
import torchvision.models as models
import torch.nn.functional as F
import json, time, datetime
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
networkAlexnet = models.alexnet(pretrained=True)


In [ ]:
print(networkAlexnet)

In [ ]:
import torchsummary as ts

ts.summary(networkAlexnet.to('cuda'), (3, 224, 224))

In [8]:
import requests

response = requests.get("https://raw.githubusercontent.com/valbarriere/CC6204-Deep-Learning/master/Additional_Material/utils/labels.txt") # https://git.io/JJkYN
labels = response.text.split("\n")

In [ ]:
print(labels)

In [ ]:
!wget https://raw.githubusercontent.com/valbarriere/CC6204-Deep-Learning/master/Additional_Material/utils/elephant.jpg -O elephant.jpg
# https://e0.pxfuel.com/wallpapers/264/159/desktop-wallpaper-elephant-yellow-field-tanzania-elephant-yellow.jpg

In [ ]:
img = Image.open('elephant.jpg')
img

In [ ]:
networkAlexnet.eval()

def transform_image(name):
  transf = T.Compose([T.Resize(255),
                      T.CenterCrop(224),
                      T.ToTensor(),
                      T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
  image = Image.open(name)
  # Size (3, 224, 224) => (1, 3, 224, 224)
  return transf(image).unsqueeze(0) 

def predict(network, name):
  tensor = transform_image(name).to('cuda')
  pred = network(tensor)
  output = F.softmax(pred, dim=-1)
  conf,val = output.max(1)
  index = str(val.item())
  o = output.detach().cpu().numpy()
  print([(l, labels[k]) for k, l in zip(np.argsort(o[0])[-5:][::-1], np.sort(o[0])[-5:][::-1])])
  return o, dict({'conf':conf,'class':labels[val.item()]})

o,d = predict(networkAlexnet.to('cuda'), 'elephant.jpg')
print(d)

plt.plot(o[0])
plt.show()

In [ ]:
networkVGG11 = models.vgg11(pretrained=True)
networkVGG11.eval()


In [ ]:
o,d = predict(networkVGG11.to('cuda'), 'elephant.jpg')
plt.plot(o[0])
plt.show()
print(d)

In [ ]:
networkGooglenet = models.googlenet(pretrained = True)
networkGooglenet.eval()

In [ ]:
o,d = predict(networkGooglenet.to('cuda'), 'elephant.jpg')

plt.plot(o[0])
plt.show()

print(d)

In [ ]:
import torchsummary as ts

ts.summary(networkGooglenet, (3, 224, 224))

In [ ]:
networkResnet = models.resnet18(pretrained=True)
networkResnet.eval()

print(networkResnet)

In [ ]:
o,d = predict(networkResnet.to('cuda'), 'elephant.jpg')

plt.plot(o[0])
plt.show()

print(d)